Mestrado em Modelagem Matematica da Informacao
----------------------------------------------
Disciplina: Modelagem e Mineracao de Dados
------------------------------------------

Master Program - Mathematical Modeling of Information
-----------------------------------------------------
Course: Data Mining and Modeling
--------------------------------

Professor: Renato Rocha Souza
-----------------------------

### Accessing and Manipulating Data in different formats

In [1]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd

In [2]:
datapath = "../datasets/"

## Acessing CSV Files

In [3]:
csvfile = "beatles-diskography.csv"
datafile = os.path.join(datapath, csvfile)

#### Using pure Python:

In [5]:
data = []
with open(datafile, "r") as f:
    keys = f.readline().split(',')
    keys = [k.strip() for k in keys]
    for i in range(5):
        values = f.readline().split(',')
        values = [v.strip() for v in values]
        d = dict(zip(keys,values))
        data.append(d)
data[0:2]

[{'BPI Certification': 'Gold',
  'Label': 'Parlophone(UK)',
  'RIAA Certification': 'Platinum',
  'Released': '22 March 1963',
  'Title': 'Please Please Me',
  'UK Chart Position': '1',
  'US Chart Position': '-'},
 {'BPI Certification': 'Platinum',
  'Label': 'Parlophone(UK)',
  'RIAA Certification': 'Gold',
  'Released': '22 November 1963',
  'Title': 'With the Beatles',
  'UK Chart Position': '1',
  'US Chart Position': '-'}]

#### Using CSV module:

https://docs.python.org/2/library/csv.html  

In [6]:
import csv

In [8]:
with open(datafile, "r") as f:
    #data2 = csv.reader(f)
    data2 = csv.DictReader(f)
    for row in data2:
        print(row)

{'UK Chart Position': '1', 'RIAA Certification': 'Platinum', 'Title': 'Please Please Me', 'Released': '22 March 1963', 'US Chart Position': '-', 'BPI Certification': 'Gold', 'Label': 'Parlophone(UK)'}
{'UK Chart Position': '1', 'RIAA Certification': 'Gold', 'Title': 'With the Beatles', 'Released': '22 November 1963', 'US Chart Position': '-', 'BPI Certification': 'Platinum', 'Label': 'Parlophone(UK)'}
{'UK Chart Position': '-', 'RIAA Certification': '', 'Title': 'Beatlemania! With the Beatles', 'Released': '25 November 1963', 'US Chart Position': '-', 'BPI Certification': '', 'Label': 'Capitol(CAN)'}
{'UK Chart Position': '-', 'RIAA Certification': '', 'Title': 'Introducing... The Beatles', 'Released': '10 January 1964', 'US Chart Position': '2', 'BPI Certification': '', 'Label': 'Vee-Jay(US)'}
{'UK Chart Position': '-', 'RIAA Certification': '5xPlatinum', 'Title': 'Meet the Beatles!', 'Released': '20 January 1964', 'US Chart Position': '1', 'BPI Certification': '', 'Label': 'Capitol(U

#### Using Pandas:

In [9]:
df_csv = pd.read_csv(datafile)
df_csv.head()
#df_csv.info()

,Title,Released,Label,UK Chart Position,US Chart Position,BPI Certification,RIAA Certification
0,Please Please Me,22 March 1963,Parlophone(UK),1,-,Gold,Platinum
1,With the Beatles,22 November 1963,Parlophone(UK),1,-,Platinum,Gold
2,Beatlemania! With the Beatles,25 November 1963,Capitol(CAN),-,-,NaN,NaN
3,Introducing... The Beatles,10 January 1964,Vee-Jay(US),-,2,NaN,NaN
4,Meet the Beatles!,20 January 1964,Capitol(US),-,1,NaN,5xPlatinum


## Manipulating Zip Files

https://docs.python.org/2/library/zipfile.html  

In [10]:
from zipfile import ZipFile

In [11]:
zipfile = "2013_ERCOT_Hourly_Load_Data"

In [12]:
with ZipFile('{0}.zip'.format(os.path.join(datapath,zipfile)), 'r') as myzip:
    myzip.extractall()

In [13]:
myzip.filename

'../datasets/2013_ERCOT_Hourly_Load_Data.zip'

## Acessing Excel Files

https://pypi.python.org/pypi/xlrd  

In [14]:
import xlrd

In [15]:
datafile = "2013_ERCOT_Hourly_Load_Data.xls"

In [16]:
workbook = xlrd.open_workbook(os.path.join(datapath,datafile))
sheet = workbook.sheet_by_index(0)

In [17]:
sheet_data = [[sheet.cell_value(r, col) for col in range(sheet.ncols)] for r in range(sheet.nrows)]
sheet_data[:2]

[['Hour_End',
  'COAST',
  'EAST',
  'FAR_WEST',
  'NORTH',
  'NORTH_C',
  'SOUTHERN',
  'SOUTH_C',
  'WEST',
  'ERCOT'],
 [41275.041666666664,
  7606.263544000012,
  1073.892438,
  1411.7505669999982,
  784.9781659999992,
  10369.094390000051,
  2206.6750770000012,
  4368.490945000006,
  882.9319009999975,
  28704.077028000065]]

In [19]:
print("Number of rows in the sheet:"),
print(sheet.nrows)

print("Type of data in cell (row 3, col 2):"), 
print(sheet.cell_type(3, 2))

print("Value in cell (row 3, col 2):", )
print(sheet.cell_value(3, 2))

print("Get a slice of values in column 3, from rows 1-3:",)
print(sheet.col_values(3, start_rowx=1, end_rowx=4))

Number of rows in the sheet:
7296
Type of data in cell (row 3, col 2):
2
Value in cell (row 3, col 2):
1036.0886969999988
Get a slice of values in column 3, from rows 1-3:
[1411.7505669999982, 1403.4722870000019, 1395.053150000001]


In [20]:
coast = sheet.col_values(1, start_rowx=1)
data3 = {}    
data3['maxvalue'] = max(coast)
data3['minvalue'] = min(coast)
data3['avgcoast'] = np.mean(coast)

rowmax = coast.index(max(coast))+1
rowmin = coast.index(min(coast))+1

data3['maxtime'] = xlrd.xldate_as_tuple(sheet.cell_value(rowmax,0), 0)
data3['mintime'] = xlrd.xldate_as_tuple(sheet.cell_value(rowmin,0), 0)

In [21]:
data3

{'avgcoast': 10976.933460679784,
 'maxtime': (2013, 8, 13, 17, 0, 0),
 'maxvalue': 18779.025510000003,
 'mintime': (2013, 2, 3, 4, 0, 0),
 'minvalue': 6602.113898999982}

#### Using Pandas

http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.excel.read_excel.html  

In [22]:
pd_excel = pd.read_excel(os.path.join(datapath,datafile))
pd_excel.head()

,Hour_End,COAST,EAST,FAR_WEST,NORTH,NORTH_C,SOUTHERN,SOUTH_C,WEST,ERCOT
0,2013-01-01 01:00:00,7606.263544,1073.892438,1411.750567,784.978166,10369.094390,2206.675077,4368.490945,882.931901,28704.077028
1,2013-01-01 02:00:00,7388.082714,1035.021938,1403.472287,776.307387,10152.358518,2159.733208,4233.587967,872.404750,28020.968769
2,2013-01-01 03:00:00,7178.867878,1036.088697,1395.053150,768.125748,9988.051418,2065.114706,4082.862860,868.853938,27383.018395
3,2013-01-01 04:00:00,7038.822581,1032.648841,1395.508820,770.937969,9946.658655,1990.903699,4010.489608,865.701201,27051.671374
4,2013-01-01 05:00:00,6990.857940,1042.823044,1401.216842,779.089313,10096.664190,1954.807585,4038.655997,879.924249,27184.039160


## Accessing HTML

https://docs.python.org/3/library/urllib.html  
http://www.crummy.com/software/BeautifulSoup/  
http://docs.python-requests.org/en/latest/  
http://lxml.de/  
https://docs.python.org/2/library/getpass.html  

In [26]:
import urllib
from bs4 import BeautifulSoup as bs
import requests
import lxml.html
import getpass

In [36]:
url = "http://oglobo.globo.com"
html_txt = urllib.request.urlopen(url).read()
soup = bs(html_txt, "html.parser")
for line in soup.find_all('a'):
    print(line.get('href'))

http://oglobo.globo.com/
http://memoria.oglobo.globo.com
http://acervo.oglobo.globo.com
http://oglobo.globo.com/ela/
http://kogut.oglobo.globo.com
http://rioshow.oglobo.globo.com/
http://extra.globo.com
http://classificados.globo.com
https://clubesoumaisrio.oglobo.globo.com/?link=og-menu-sup
http://www.centraldoassinante.oglobo.com.br
http://oglobo.globo.com/
http://oglobo.globo.com/principios-editoriais/
http://facebook.com/JornalOGlobo
https://twitter.com/JornalOGlobo
https://plus.google.com/u/0/+JornalOGlobo/posts

https://seguro.oglobo.com.br/assinatura/Default.aspx?id_parc=26&utm_source=venda&utm_medium=botaooglobo&utm_campaign=footer
http://oglobo.globo.com/
http://oglobo.globo.com/rio/
http://oglobo.globo.com/rio/
http://oglobo.globo.com/rio/
http://oglobo.globo.com/rio/ancelmo/
http://oglobo.globo.com/blogs/blog_gente_boa/
http://oglobo.globo.com/rio/carnaval/
http://oglobo.globo.com/rio/bairros/
http://oglobo.globo.com/rio/design-rio/
http://oglobo.globo.com/eu-reporter/
http:

Another approach (some say it is faster)  
http://lxml.de/tutorial.html  

In [38]:
url = "http://www.hotashtanga.com/p/letoltesek-downloads.html"
html_txt = urllib.request.urlopen(url).read()
dom =  lxml.html.fromstring(html_txt)
for link in dom.xpath('//a/@href'): # select the url in href for all a tags(links)
    print(link)

http://www.hotashtanga.com/
http://www.youtube.com/GaurangaDas
https://drive.google.com/file/d/0B0t9QIGAtRQ5Z2VrVl9iYlI0Q0U/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5RE9BYkdGTlU2MHc/edit?usp=sharing
http://brihaspati.net/downloads/2009_07_08_Patandzsali_beavatas.mp3
https://drive.google.com/file/d/0B0t9QIGAtRQ5Y2kyb0NpWTgtQnM/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5MnFlOXNmN19UMlU/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5ellsNDNyOUFEQjA/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5bVV4TlNzLTFDZ00/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5Ukdoa3V5eE9IM00/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5MW1OSjdCNjBUUGc/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5RVF2SFNKNnptUzQ/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5eWxGUDU5ZzZieDg/edit?usp=sharing
https://drive.google.com/file/d/0B0t9QIGAtRQ5aTN1MkdtNnBISXc/edit?usp=sharing
https://dri

Using Requests, for more complex tasks  

http://docs.python-requests.org/en/master/user/quickstart/  
http://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html  

In [39]:
p = getpass.getpass()
r = requests.get('https://api.github.com/user', auth=('rsouza', p))
r.status_code

········


200

In [40]:
r.headers['content-type']

'application/json; charset=utf-8'

In [41]:
r.encoding

'utf-8'

In [42]:
r.text

'{"login":"rsouza","id":604914,"avatar_url":"https://avatars.githubusercontent.com/u/604914?v=3","gravatar_id":"","url":"https://api.github.com/users/rsouza","html_url":"https://github.com/rsouza","followers_url":"https://api.github.com/users/rsouza/followers","following_url":"https://api.github.com/users/rsouza/following{/other_user}","gists_url":"https://api.github.com/users/rsouza/gists{/gist_id}","starred_url":"https://api.github.com/users/rsouza/starred{/owner}{/repo}","subscriptions_url":"https://api.github.com/users/rsouza/subscriptions","organizations_url":"https://api.github.com/users/rsouza/orgs","repos_url":"https://api.github.com/users/rsouza/repos","events_url":"https://api.github.com/users/rsouza/events{/privacy}","received_events_url":"https://api.github.com/users/rsouza/received_events","type":"User","site_admin":false,"name":"Renato Rocha Souza","company":"FGV","blog":"emap.fgv.br","location":"Rio de Janeiro","email":"rsouza.fgv@gmail.com","hireable":null,"bio":"https:

In [43]:
r.json()

{'avatar_url': 'https://avatars.githubusercontent.com/u/604914?v=3',
 'bio': 'https://www.linkedin.com/in/renato-rocha-souza-157153',
 'blog': 'emap.fgv.br',
 'collaborators': 0,
 'company': 'FGV',
 'created_at': '2011-02-07T15:46:45Z',
 'disk_usage': 100731,
 'email': 'rsouza.fgv@gmail.com',
 'events_url': 'https://api.github.com/users/rsouza/events{/privacy}',
 'followers': 22,
 'followers_url': 'https://api.github.com/users/rsouza/followers',
 'following': 5,
 'following_url': 'https://api.github.com/users/rsouza/following{/other_user}',
 'gists_url': 'https://api.github.com/users/rsouza/gists{/gist_id}',
 'gravatar_id': '',
 'hireable': None,
 'html_url': 'https://github.com/rsouza',
 'id': 604914,
 'location': 'Rio de Janeiro',
 'login': 'rsouza',
 'name': 'Renato Rocha Souza',
 'organizations_url': 'https://api.github.com/users/rsouza/orgs',
 'owned_private_repos': 0,
 'plan': {'collaborators': 0,
  'name': 'free',
  'private_repos': 0,
  'space': 976562499},
 'private_gists': 0,

## Accessing JSON

http://json.org/  
https://docs.python.org/2/library/json.html  
https://docs.python.org/2/library/urlparse.html  

In [45]:
import json
#import urlparse

In [50]:
base_url = "http://musicbrainz.org/ws/2/"
artist_url =  urllib.parse.urljoin(base_url, 'artist/')
query_type = {"simple": {},
              "atr": {"inc": "aliases+tags+ratings"},
              "aliases": {"inc": "aliases"},
              "releases": {"inc": "releases"}}

In [51]:
def query_site(url, params, uid="", fmt="json"):
    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print("requesting", r.url)

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()

In [52]:
def query_by_name(url, params, name):
    params["query"] = "artist:" + name
    return query_site(url, params)

In [54]:
def pretty_print(data, indent=4):
    if type(data) == dict:
        print(json.dumps(data, indent=indent, sort_keys=True))
    else:
        print(data)

In [65]:
results = query_by_name(artist_url, query_type["simple"], "Nirvana")
pretty_print(results)

requesting http://musicbrainz.org/ws/2/artist/?fmt=json&query=artist%3ANirvana
{
    "artists": [
        {
            "aliases": [
                {
                    "begin-date": null,
                    "end-date": null,
                    "locale": null,
                    "name": "Nirvana US",
                    "primary": null,
                    "sort-name": "Nirvana US",
                    "type": null
                }
            ],
            "area": {
                "id": "489ce91b-6658-3307-9877-795b68554c98",
                "name": "United States",
                "sort-name": "United States"
            },
            "begin-area": {
                "id": "a640b45c-c173-49b1-8030-973603e895b5",
                "name": "Aberdeen",
                "sort-name": "Aberdeen"
            },
            "country": "US",
            "disambiguation": "90s US grunge band",
            "id": "5b11f4ce-a62d-471e-81fc-a69a8278c7da",
            "life-span": {
           

In [66]:
results.keys()

dict_keys(['created', 'count', 'offset', 'artists'])

In [67]:
artist_id = results["artists"][1]["id"]
artist_id

'9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6'

In [68]:
print("\nARTIST:")
pretty_print(results["artists"][0])


ARTIST:
{
    "aliases": [
        {
            "begin-date": null,
            "end-date": null,
            "locale": null,
            "name": "Nirvana US",
            "primary": null,
            "sort-name": "Nirvana US",
            "type": null
        }
    ],
    "area": {
        "id": "489ce91b-6658-3307-9877-795b68554c98",
        "name": "United States",
        "sort-name": "United States"
    },
    "begin-area": {
        "id": "a640b45c-c173-49b1-8030-973603e895b5",
        "name": "Aberdeen",
        "sort-name": "Aberdeen"
    },
    "country": "US",
    "disambiguation": "90s US grunge band",
    "id": "5b11f4ce-a62d-471e-81fc-a69a8278c7da",
    "life-span": {
        "begin": "1988-01",
        "end": "1994-04-05",
        "ended": true
    },
    "name": "Nirvana",
    "score": "100",
    "sort-name": "Nirvana",
    "tags": [
        {
            "count": 1,
            "name": "punk"
        },
        {
            "count": 0,
            "name": "legendary"

In [69]:
artist_data = query_site(artist_url, query_type["releases"], artist_id)
releases = artist_data["releases"]
print("\nONE RELEASE:")
pretty_print(releases[0], indent=2)

requesting http://musicbrainz.org/ws/2/artist/9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6?inc=releases&fmt=json

ONE RELEASE:
{
  "barcode": null,
  "country": "GB",
  "date": "1969",
  "disambiguation": "",
  "id": "0b44cb36-550a-491d-bfd9-8751271f9de7",
  "packaging": null,
  "packaging-id": null,
  "quality": "normal",
  "release-events": [
    {
      "area": {
        "disambiguation": "",
        "id": "8a754a16-0027-3a29-b6d7-2b40ea0481ed",
        "iso-3166-1-codes": [
          "GB"
        ],
        "name": "United Kingdom",
        "sort-name": "United Kingdom"
      },
      "date": "1969"
    }
  ],
  "status": "Official",
  "status-id": "4e304316-386d-3409-af2e-78857eec5cfe",
  "text-representation": {
    "language": "eng",
    "script": "Latn"
  },
  "title": "To Markos III"
}


In [70]:
releases

[{'barcode': None,
  'country': 'GB',
  'date': '1969',
  'disambiguation': '',
  'id': '0b44cb36-550a-491d-bfd9-8751271f9de7',
  'packaging': None,
  'packaging-id': None,
  'quality': 'normal',
  'release-events': [{'area': {'disambiguation': '',
     'id': '8a754a16-0027-3a29-b6d7-2b40ea0481ed',
     'iso-3166-1-codes': ['GB'],
     'name': 'United Kingdom',
     'sort-name': 'United Kingdom'},
    'date': '1969'}],
  'status': 'Official',
  'status-id': '4e304316-386d-3409-af2e-78857eec5cfe',
  'text-representation': {'language': 'eng', 'script': 'Latn'},
  'title': 'To Markos III'},
 {'barcode': None,
  'country': 'GB',
  'date': '1992',
  'disambiguation': '',
  'id': '223a7069-3d54-40a1-9e57-fc1339f38aee',
  'packaging': None,
  'packaging-id': None,
  'quality': 'normal',
  'release-events': [{'area': {'disambiguation': '',
     'id': '8a754a16-0027-3a29-b6d7-2b40ea0481ed',
     'iso-3166-1-codes': ['GB'],
     'name': 'United Kingdom',
     'sort-name': 'United Kingdom'},
    

In [71]:
release_titles = [r["title"] for r in releases]
print("\nALL TITLES:")
for t in release_titles:
        print(t)


ALL TITLES:
To Markos III
Travelling on a Cloud
Songs Of Love And Praise
Songs of Love and Praise
Songs of Love and Praise
Secret Theatre
The Story of Simon Simopath
Me And My Friend
All of Us
The Story of Simon Simopath
To Markos III
Chemistry
Local Anaesthetic
Orange & Blue
Pentecost Hotel
Black Flower
All of Us


#### Using Pandas

http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader  

In [72]:
df_json = pd.read_json('https://api.github.com/repos/pydata/pandas/issues?per_page=5')
df_json

,assignee,assignees,body,closed_at,comments,comments_url,created_at,events_url,html_url,id,...,locked,milestone,number,pull_request,repository_url,state,title,updated_at,url,user
0,NaN,[],"Hello,\r\n\r\nI have a column of datetimes whi...",NaT,0,https://api.github.com/repos/pydata/pandas/iss...,2016-07-13 17:02:17,https://api.github.com/repos/pydata/pandas/iss...,https://github.com/pydata/pandas/issues/13644,165371687,...,False,None,13644,NaN,https://api.github.com/repos/pydata/pandas,open,QuarterEnd Offset seems to not work with datet...,2016-07-13 17:02:17,https://api.github.com/repos/pydata/pandas/iss...,{'url': 'https://api.github.com/users/thequack...
1,NaN,[],- [x] passes ``git diff upstream/master | fla...,NaT,1,https://api.github.com/repos/pydata/pandas/iss...,2016-07-13 16:20:52,https://api.github.com/repos/pydata/pandas/iss...,https://github.com/pydata/pandas/pull/13643,165362966,...,False,{'url': 'https://api.github.com/repos/pydata/p...,13643,{'url': 'https://api.github.com/repos/pydata/p...,https://api.github.com/repos/pydata/pandas,open,CLN: Fix compile time warnings,2016-07-13 19:01:21,https://api.github.com/repos/pydata/pandas/iss...,{'url': 'https://api.github.com/users/yui-knk'...
2,NaN,[],`DataFrame.rename_axis() ` is documented here:...,NaT,0,https://api.github.com/repos/pydata/pandas/iss...,2016-07-13 15:18:00,https://api.github.com/repos/pydata/pandas/iss...,https://github.com/pydata/pandas/issues/13642,165347749,...,False,{'url': 'https://api.github.com/repos/pydata/p...,13642,NaN,https://api.github.com/repos/pydata/pandas,open,Missing DataFrame.rename_axis in API Table of ...,2016-07-13 18:03:13,https://api.github.com/repos/pydata/pandas/iss...,"{'url': 'https://api.github.com/users/Dr-Irv',..."
3,NaN,[],Closes #13570\r\n\r\n - [x] tests added / pass...,NaT,3,https://api.github.com/repos/pydata/pandas/iss...,2016-07-13 15:01:57,https://api.github.com/repos/pydata/pandas/iss...,https://github.com/pydata/pandas/pull/13641,165343748,...,False,{'url': 'https://api.github.com/repos/pydata/p...,13641,{'url': 'https://api.github.com/repos/pydata/p...,https://api.github.com/repos/pydata/pandas,open,BF(TST): allow AttributeError being raised (in...,2016-07-13 18:07:52,https://api.github.com/repos/pydata/pandas/iss...,{'url': 'https://api.github.com/users/yarikopt...
4,NaN,[],https://travis-ci.org/pydata/pandas/jobs/14434...,NaT,6,https://api.github.com/repos/pydata/pandas/iss...,2016-07-13 10:26:03,https://api.github.com/repos/pydata/pandas/iss...,https://github.com/pydata/pandas/issues/13639,165287855,...,False,{'url': 'https://api.github.com/repos/pydata/p...,13639,NaN,https://api.github.com/repos/pydata/pandas,open,DOC/BLD: IPython 5.0 breaks doc building,2016-07-13 15:00:19,https://api.github.com/repos/pydata/pandas/iss...,{'url': 'https://api.github.com/users/jreback'...


## Accessing XML

https://docs.python.org/2/library/urllib2.html

https://docs.python.org/2/library/xml.etree.elementtree.html

http://lxml.de/

http://www.w3schools.com/xml/xml_examples.asp

In [73]:
from xml.dom import minidom
#from xml.etree import ElementTree as ET
from lxml import etree as ET #Supports xpath syntax
from collections import defaultdict
from pprint import pprint

In [87]:
url = 'http://www.w3schools.com/xml/cd_catalog.xml'
xml_page = urllib.request.urlopen(url).read()
e = ET.XML(xml_page)

In [88]:
def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k:v[0] if len(v) == 1 else v for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v) for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
              d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [89]:
pprint(etree_to_dict(e))

{'CATALOG': {'CD': [{'ARTIST': 'Bob Dylan',
                     'COMPANY': 'Columbia',
                     'COUNTRY': 'USA',
                     'PRICE': '10.90',
                     'TITLE': 'Empire Burlesque',
                     'YEAR': '1985'},
                    {'ARTIST': 'Bonnie Tyler',
                     'COMPANY': 'CBS Records',
                     'COUNTRY': 'UK',
                     'PRICE': '9.90',
                     'TITLE': 'Hide your heart',
                     'YEAR': '1988'},
                    {'ARTIST': 'Dolly Parton',
                     'COMPANY': 'RCA',
                     'COUNTRY': 'USA',
                     'PRICE': '9.90',
                     'TITLE': 'Greatest Hits',
                     'YEAR': '1982'},
                    {'ARTIST': 'Gary Moore',
                     'COMPANY': 'Virgin records',
                     'COUNTRY': 'UK',
                     'PRICE': '10.20',
                     'TITLE': 'Still got the blues',
                

In [90]:
raw_page = urllib.request.urlopen(url)
doc = minidom.parse(raw_page)

def findTextnodes(nodeList):
    for subnode in nodeList:
        if subnode.nodeType == subnode.ELEMENT_NODE:
            print("Element node: " + subnode.tagName)
            findTextnodes(subnode.childNodes)
        elif subnode.nodeType == subnode.TEXT_NODE:
            print("text node:" + subnode.data)
            
findTextnodes(doc.childNodes)

Element node: CATALOG
text node:
	
Element node: CD
text node:
		
Element node: TITLE
text node:Empire Burlesque
text node:
		
Element node: ARTIST
text node:Bob Dylan
text node:
		
Element node: COUNTRY
text node:USA
text node:
		
Element node: COMPANY
text node:Columbia
text node:
		
Element node: PRICE
text node:10.90
text node:
		
Element node: YEAR
text node:1985
text node:
	
text node:
	
Element node: CD
text node:
		
Element node: TITLE
text node:Hide your heart
text node:
		
Element node: ARTIST
text node:Bonnie Tyler
text node:
		
Element node: COUNTRY
text node:UK
text node:
		
Element node: COMPANY
text node:CBS Records
text node:
		
Element node: PRICE
text node:9.90
text node:
		
Element node: YEAR
text node:1988
text node:
	
text node:
	
Element node: CD
text node:
		
Element node: TITLE
text node:Greatest Hits
text node:
		
Element node: ARTIST
text node:Dolly Parton
text node:
		
Element node: COUNTRY
text node:USA
text node:
		
Element node: COMPANY
text node:RCA
text 

In [99]:
root = ET.fromstring(xml_page)
cdtags = root.xpath('//CD/TITLE')
for cd in cdtags:
    print(cd.text)

Empire Burlesque
Hide your heart
Greatest Hits
Still got the blues
Eros
One night only
Sylvias Mother
Maggie May
Romanza
When a man loves a woman
Black angel
1999 Grammy Nominees
For the good times
Big Willie style
Tupelo Honey
Soulsville
The very best of
Stop
Bridge of Spies
Private Dancer
Midt om natten
Pavarotti Gala Concert
The dock of the bay
Picture book
Red
Unchain my heart


In [108]:
for cd in root.findall('CD'):
    title = cd.find('TITLE').text
    artist = cd.find('ARTIST').text
    country = cd.find('COUNTRY').text
    print('{}\t{}\t{}'.format(artist, country, title))

Bob Dylan	USA	Empire Burlesque
Bonnie Tyler	UK	Hide your heart
Dolly Parton	USA	Greatest Hits
Gary Moore	UK	Still got the blues
Eros Ramazzotti	EU	Eros
Bee Gees	UK	One night only
Dr.Hook	UK	Sylvias Mother
Rod Stewart	UK	Maggie May
Andrea Bocelli	EU	Romanza
Percy Sledge	USA	When a man loves a woman
Savage Rose	EU	Black angel
Many	USA	1999 Grammy Nominees
Kenny Rogers	UK	For the good times
Will Smith	USA	Big Willie style
Van Morrison	UK	Tupelo Honey
Jorn Hoel	Norway	Soulsville
Cat Stevens	UK	The very best of
Sam Brown	UK	Stop
T'Pau	UK	Bridge of Spies
Tina Turner	UK	Private Dancer
Kim Larsen	EU	Midt om natten
Luciano Pavarotti	UK	Pavarotti Gala Concert
Otis Redding	USA	The dock of the bay
Simply Red	EU	Picture book
The Communards	UK	Red
Joe Cocker	USA	Unchain my heart


In [109]:
cds = []
for cd in root.findall('./CD'):
        data = {}
        data["TITLE"] = cd.find('./TITLE').text
        data["ARTIST"] = cd.find('./ARTIST').text
        data["COUNTRY"] = cd.find('./COUNTRY').text
        data["PRICE"] = cd.find('./PRICE').text
        data["YEAR"] = cd.find('./YEAR').text
        cds.append(data)
cds[0:3]

[{'ARTIST': 'Bob Dylan',
  'COUNTRY': 'USA',
  'PRICE': '10.90',
  'TITLE': 'Empire Burlesque',
  'YEAR': '1985'},
 {'ARTIST': 'Bonnie Tyler',
  'COUNTRY': 'UK',
  'PRICE': '9.90',
  'TITLE': 'Hide your heart',
  'YEAR': '1988'},
 {'ARTIST': 'Dolly Parton',
  'COUNTRY': 'USA',
  'PRICE': '9.90',
  'TITLE': 'Greatest Hits',
  'YEAR': '1982'}]

## Acessing Databases

### Acessing SQLite

https://docs.python.org/2/library/sqlite3.html

In [110]:
import sqlite3
conn = sqlite3.connect('example.db')

In [111]:
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE stocks
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [112]:
conn = sqlite3.connect('example.db')
c = conn.cursor()

In [114]:
t = ('RHAT',) #tuple with just one element
c.execute('SELECT * FROM stocks WHERE symbol=?', t)
print(c.fetchone())

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


In [115]:
# Larger example that inserts many records at a time
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
c.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)

In [116]:
for row in c.execute('SELECT * FROM stocks ORDER BY price'):
        print(row)

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)


### Acessing MySQL  

https://pypi.python.org/pypi/PyMySQL  

In [117]:
import pymysql

In [ ]:
p = getpass.getpass()
conn = pymysql.connect(host='localhost', port=3306, user='rsouza', passwd=p, db='mysql')
cur = conn.cursor()

In [ ]:
cur.execute("show databases;")
for r in cur.fetchall():
   print(r)

In [ ]:
cur.execute("use mysql;")
cur.execute("show tables;")
for r in cur.fetchall():
   print(r)

In [ ]:
cur.execute("SELECT * FROM mysql.user;")# LIMIT 10;")
for r in cur.fetchall():
   print(r)

#### Using Pandas

In [ ]:
import pandas.io.sql as psql

In [ ]:
df_mysql = psql.read_sql('select * from reuters.Frequency;', con=conn)

In [ ]:
df_mysql.head()

In [ ]:
cur.close()
conn.close()

### Acessing PostgreSQL 

In [120]:
import psycopg2

In [125]:
p = getpass.getpass()
try:
    conn = psycopg2.connect("dbname='dbamv' user='rsouza' host='localhost' password='{}'".format(p))
except:
    print("I am unable to connect to the database")
cur = conn.cursor()

········


### Acessing Mongo DB

http://api.mongodb.org/python/current/tutorial.html

In [128]:
from pymongo import MongoClient

ImportError: No module named 'pymongo'

In [ ]:
client = MongoClient()
#client = MongoClient('localhost', 27017)
db = client.test_database #acessa ou cria o banco

In [ ]:
post = {"author": "Mike",
        "text": "My first blog post!",
        "tags": ["mongodb", "python", "pymongo"],
        "date": datetime.datetime.utcnow()}

In [ ]:
posts = db.posts # cria a coleção
post_id = posts.insert(post) #insere dados
post_id

In [ ]:
db.collection_names()

In [ ]:
posts.find_one()

In [ ]:
new_posts = [{"author": "Mike",
              "text": "Another post!",
              "tags": ["bulk", "insert"],
              "date": datetime.datetime(2009, 11, 12, 11, 14)},
             {"author": "Eliot",
              "title": "MongoDB is fun",
              "text": "and pretty easy too!",
              "date": datetime.datetime(2009, 11, 10, 10, 45)}]

In [ ]:
posts.insert(new_posts)

In [ ]:
posts.find_one({"author": "Mike"})

In [ ]:
#for post in posts.find({"author": "Mike"}):
for post in posts.find():
    print(post)

In [ ]:
posts.count()

#### Using Pandas

In [ ]:
example_pd = posts.find()
df_pandas =  pd.DataFrame(list(example_pd))
df_pandas

## Accessing Feeds RSS

http://www.feedparser.org/  
http://docs.python.org/library/re.html  
http://www.pythonware.com/library/pil/handbook/index.htm  

In [ ]:
import feedparser

In [ ]:
#d = feedparser.parse('http://g1.globo.com/dynamo/rss2.xml')
d = feedparser.parse('http://rss.nytimes.com/services/xml/rss/nyt/InternationalHome.xml')

In [ ]:
print d['feed']['title']
print d['feed']['link']
print d.feed.subtitle
print len(d['entries'])

In [ ]:
print d['entries'][0]['title'] 
print d.entries[0]['link']

In [ ]:
print d.headers

In [ ]:
for post in d.entries:
    print post.title + ": " + post.link + "\n"

## Acessando Twitter

https://github.com/bear/python-twitter  
Go to http://twitter.com/apps/new to create an app and get these items  
See https://dev.twitter.com/docs/auth/oauth for more information on Twitter's OAuth implementation  
https://dev.twitter.com/rest/reference/get/account/verify_credentials  

In [ ]:
import twitter

In [ ]:
with open('twitter_tokens.txt', 'r') as twitter_tokens:
    tokens = twitter_tokens.read().split(',')
consumer_key = tokens[0].strip()
consumer_secret = tokens[1].strip()
access_token = tokens[2].strip()
access_token_secret = tokens[3].strip()

Construct, sign, and open a twitter request using the hard-coded credentials above.

In [ ]:
api = twitter.Api(consumer_key=consumer_key, consumer_secret=consumer_secret, 
                  access_token_key=access_token, access_token_secret=access_token_secret)

In [ ]:
print api.VerifyCredentials()

In [ ]:
users = api.GetFriends()
print [u.name for u in users]

In [ ]:
followers = api.GetFollowers()
print [u.screen_name for u in followers]

In [ ]:
mentions = api.GetMentions()
print [m.GetText() for m in mentions]

In [ ]:
statuses = api.GetUserTimeline(screen_name='TIME')
print [s.text for s in statuses]